In [ ]:
# Lab-04-2 Loading Data

# Data in Real World
# 복잡한 머신러닝 모델을 학습하려면 엄청난 양의 데이터가 필요하다!
# 대부분 데이터셋은 적어도 수십만 개의 데이터를 제공한다.
# 엄청난 양의 데이터를 한번에 학습시킬 수 없다! / 너무 느리다. / 하드웨어적으로 불가능하다.
# 그렇다면 일부분의 데이터로만 학습하면 어떨까?

# 그래서 Minibatch Gradient Descent
# 전체 데이터를 균일하게 나눠서 학습하자!

# 한번의 Gradient Descent하지 않고, 모델의 cost를 계산할때 전체 data를 사용하지 않아 업데이트를 좀 더 빠르게 할 수 있다.
# 하지만, 전체 데이터를 쓰지 않아서 잘못된 방향으로 업데이트를 할 수도 있다.





: 

In [6]:
import torch
from torch.autograd import Variable

x_train = Variable(torch.FloatTensor([[73, 80, 75],
                                      [93, 88, 93],
                                      [89, 91, 90],
                                      [96, 88, 100],
                                      [73, 66, 70]]))
y_train = Variable(torch.FloatTensor([[152],
                                      [185],
                                      [180],
                                      [196],
                                      [142]]))

class MultivariateLinearRegressionModel(torch.nn.Module):
    def __init__(self):
        super(MultivariateLinearRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(3, 1)

    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

model = MultivariateLinearRegressionModel()

criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

epochs = 20
for epoch in range(1, epochs+1):
    y_pred = model(x_train)

    # Compute and print loss
    loss = criterion(y_pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"Epoch : {epoch}\ty_pred = {y_pred.squeeze().detach()}\tLoss = {loss.item()}")

Epoch : 1	y_pred = tensor([-12.5408,  -8.8527, -11.9634,  -7.0743,  -5.7500])	Loss = 32914.3515625
Epoch : 2	y_pred = tensor([57.6540, 75.5818, 71.1968, 80.4813, 58.6613])	Loss = 10600.3076171875
Epoch : 3	y_pred = tensor([ 97.4876, 123.4957, 118.3878, 130.1664,  95.2127])	Loss = 3414.71337890625
Epoch : 4	y_pred = tensor([120.0921, 150.6853, 145.1673, 158.3609, 115.9543])	Loss = 1100.80078125
Epoch : 5	y_pred = tensor([132.9197, 166.1145, 160.3639, 174.3603, 127.7245])	Loss = 355.67254638671875
Epoch : 6	y_pred = tensor([140.1991, 174.8701, 168.9877, 183.4393, 134.4035])	Loss = 115.7247085571289
Epoch : 7	y_pred = tensor([144.3302, 179.8385, 173.8815, 188.5913, 138.1936])	Loss = 38.456207275390625
Epoch : 8	y_pred = tensor([146.6747, 182.6579, 176.6586, 191.5147, 140.3443])	Loss = 13.573675155639648
Epoch : 9	y_pred = tensor([148.0053, 184.2578, 178.2347, 193.1736, 141.5646])	Loss = 5.560627460479736
Epoch : 10	y_pred = tensor([148.7606, 185.1656, 179.1292, 194.1148, 142.2570])	Loss =

In [7]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                        [93, 88, 93],
                        [89, 91, 90],
                        [96, 88, 100],
                        [73, 66, 70]]
        self.y_data = [[152],
                        [185],
                        [180],
                        [196],
                        [142]]

    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])

        return x, y

dataset = CustomDataset()        

In [10]:
from torch.utils.data import DataLoader
import torch.nn as nn 
import torch.nn.functional as F

dataloader = DataLoader(
    dataset,
    batch_size = 2,
    shuffle = True,
)

nb_epochs = 20
for epoch in range(nb_epochs+1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Batch {}/{} Cost : {:.6f}'.format(epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item()))
        
# batch size = 2 / 각 minibatch의 크기 / 통상적으로 2의 제곱수로 설정한다.(16, 32, 64, 128, 256...)
# suffle = Epoch 마다 데이터셋을 섞어서, 데이터가 학습되는 순서를 바꾼다.
# enumerate(dataloader) : minibatch 인덱스와 데이터를 받음
# len(dataloader) : 한 epoch당 minibatch 개수


Epoch    0/20 Batch 1/3 Cost : 1.130375
Epoch    0/20 Batch 2/3 Cost : 0.542440
Epoch    0/20 Batch 3/3 Cost : 6.645627
Epoch    1/20 Batch 1/3 Cost : 3.469215
Epoch    1/20 Batch 2/3 Cost : 0.338454
Epoch    1/20 Batch 3/3 Cost : 2.030808
Epoch    2/20 Batch 1/3 Cost : 0.673827
Epoch    2/20 Batch 2/3 Cost : 4.269202
Epoch    2/20 Batch 3/3 Cost : 2.035769
Epoch    3/20 Batch 1/3 Cost : 0.376510
Epoch    3/20 Batch 2/3 Cost : 3.179954
Epoch    3/20 Batch 3/3 Cost : 2.551823
Epoch    4/20 Batch 1/3 Cost : 1.032176
Epoch    4/20 Batch 2/3 Cost : 0.479723
Epoch    4/20 Batch 3/3 Cost : 7.511059
Epoch    5/20 Batch 1/3 Cost : 1.232747
Epoch    5/20 Batch 2/3 Cost : 1.467904
Epoch    5/20 Batch 3/3 Cost : 6.093295
Epoch    6/20 Batch 1/3 Cost : 1.845562
Epoch    6/20 Batch 2/3 Cost : 2.385175
Epoch    6/20 Batch 3/3 Cost : 1.771991
Epoch    7/20 Batch 1/3 Cost : 3.486458
Epoch    7/20 Batch 2/3 Cost : 1.986333
Epoch    7/20 Batch 3/3 Cost : 1.049549
Epoch    8/20 Batch 1/3 Cost : 1.088457
